#### Simple Encoder Decoder Attention Mechanism

In [3]:
import numpy as np 
import pandas as pd 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

from keras.layers import Layer
import tensorflow as tf

#### Preparing Input Data 

In [4]:
data_path = 'dataset/amazon_cells_labelled.txt'
messages = []
labels = []

with open(data_path, 'r', encoding='utf-8') as f:
    datas = f.read().split("\n")
    for data in datas:
        if not data.strip():
            continue
        row =  data.split('\t')

        if len(row) == 2:
            messages.append(row[0])
            labels.append(int(row[1]))


print("messages : ", messages)
print("length of messages : ", len(messages))
print("labels : ", labels)
print("length of labels : ", len(labels))

messages :  ['So there is no way for me to plug it in here in the US unless I go by a converter.', 'Good case, Excellent value.', 'Great for the jawbone.', 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!', 'The mic is great.', 'I have to jiggle the plug to get it to line up right to get decent volume.', 'If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.', 'If you are Razr owner...you must have this!', 'Needless to say, I wasted my money.', 'What a waste of money and time!.', 'And the sound quality is great.', 'He was very impressed when going from the original battery to the extended battery.', 'If the two were seperated by a mere 5+ ft I started to notice excessive static and garbled sound from the headset.', 'Very good quality though', 'The design is very odd, as the ear "clip" is not very comfortable at all.', 'Highly recommend for any one who has a blue tooth phone.', 'I advise EVERYONE D

In [5]:
dataset = pd.DataFrame({'message': messages, "label": labels})
dataset.head(10)

,message,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
5,I have to jiggle the plug to get it to line up...,0
6,If you have several dozen or several hundred c...,0
7,If you are Razr owner...you must have this!,1
8,"Needless to say, I wasted my money.",0
9,What a waste of money and time!.,0


#### Tokenizing the Words and Converting it into Numerical Sequences

In [6]:
t = Tokenizer() 
t.fit_on_texts(dataset['message'])
text_matrix = t.texts_to_sequences(dataset['message'])

#### We need to know the Maximum Length of the Matrix to Pad a sequences

In [8]:
max_len = 0
for matrix in text_matrix:
    length = len(matrix)
    if length > max_len : 
        max_len = length

print("Maximum length of sequences can be: ", max_len)

Maximum length of sequences can be:  30


In [9]:
text_pad = pad_sequences(text_matrix, maxlen=32, padding='post' )

In [10]:
X_train, X_test, y_train, y_test = train_test_split(text_pad, dataset['label'], test_size=0.2, random_state=42)

In [12]:
features = 32
vocab = t.word_index
vocab_size = len(vocab)
print("dicitionary element in knowledge based ", vocab)
print("vocabulary size of the knowlefge based : ", vocab_size)

dicitionary element in knowledge based  {'the': 1, 'i': 2, 'and': 3, 'it': 4, 'is': 5, 'a': 6, 'this': 7, 'to': 8, 'phone': 9, 'my': 10, 'for': 11, 'of': 12, 'not': 13, 'with': 14, 'very': 15, 'great': 16, 'was': 17, 'on': 18, 'in': 19, 'that': 20, 'good': 21, 'have': 22, 'you': 23, 'product': 24, 'quality': 25, 'had': 26, 'headset': 27, 'works': 28, 'battery': 29, 'but': 30, 'as': 31, 'sound': 32, 'so': 33, 'are': 34, 'well': 35, 'one': 36, 'all': 37, 'use': 38, 'ear': 39, 'has': 40, 'would': 41, 'work': 42, 'from': 43, 'your': 44, 'like': 45, 'be': 46, 'me': 47, 'case': 48, 'if': 49, 'than': 50, "i've": 51, "don't": 52, 'no': 53, 'excellent': 54, 'up': 55, 'time': 56, "it's": 57, 'after': 58, 'price': 59, 'recommend': 60, 'does': 61, 'really': 62, '2': 63, 'at': 64, 'or': 65, 'best': 66, 'out': 67, 'only': 68, 'service': 69, 'get': 70, 'when': 71, 'nice': 72, "i'm": 73, 'also': 74, 'too': 75, 'just': 76, 'any': 77, 'new': 78, 'love': 79, 'these': 80, 'worked': 81, 'am': 82, 'charger'

In [13]:
#### Defining LSTM Model

In [14]:
# Now lets define the basic LSTM based model

inputs1 = Input(shape=(features, ))
x1 = Embedding(input_dim=vocab_size + 1, output_dim = 32, 
input_length = features, embeddings_regularizer = regularizers.l2(.001))(inputs1)
x1 = LSTM(100, dropout=0.3, recurrent_dropout=0.2)(x1)
outputs1 = Dense(1, activation='sigmoid')(x1)
model1  = Model(inputs1, outputs1)

c:\Users\Asus\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 32, 32)         │        60,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        53,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 113,429 (443.08 KB)

 Trainable params: 113,429 (443.08 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model1.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['accuracy'])
history = model1.fit(x=X_train, y=y_train, batch_size=100, epochs=10, verbose=1, shuffle=True, validation_split=0.2)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.4859 - loss: 0.7372 - val_accuracy: 0.4812 - val_loss: 0.7259
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4844 - loss: 0.7206 - val_accuracy: 0.5188 - val_loss: 0.7127
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4844 - loss: 0.7095 - val_accuracy: 0.5188 - val_loss: 0.7040
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5125 - loss: 0.7021 - val_accuracy: 0.5188 - val_loss: 0.6986
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5078 - loss: 0.6982 - val_accuracy: 0.5188 - val_loss: 0.6956
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5078 - loss: 0.6955 - val_accuracy: 0.5188 - val_loss: 0.6939
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5250 - loss: 0.6939 - val_accuracy: 0.5375 - val_loss: 0.6928
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5312 - loss: 0.6924 - val_accuracy: 0.5375 - val_loss: 0.6904


#### Self Attention Mechanism

In [17]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self, x):
        # x shape: (batch, timesteps, features)
        
        et = tf.squeeze(tf.tanh(tf.linalg.matmul(x, self.W) + self.b), axis=-1)  # (batch, timesteps)
        at = tf.nn.softmax(et)  # attention weights (batch, timesteps)
        at = tf.expand_dims(at, axis=-1)  # (batch, timesteps, 1)
        
        output = x * at  # (batch, timesteps, features)
        return tf.reduce_sum(output, axis=1)  # context vector

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [18]:
inputs=Input((features,))
x=Embedding(input_dim=vocab_size+1,output_dim=32,input_length=features,\
            embeddings_regularizer=regularizers.l2(.001))(inputs)
att_in=LSTM(100,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(x)
att_out=attention()(att_in)
outputs=Dense(1,activation='sigmoid',trainable=True)(att_out)
model=Model(inputs,outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 32, 32)         │        60,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 100)        │        53,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention (attention)           │ (None, 100)            │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 113,561 (443.60 KB)

 Trainable params: 113,561 (443.60 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train,y_train,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - acc: 0.5016 - loss: 0.7370 - val_acc: 0.5188 - val_loss: 0.7252
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc: 0.4922 - loss: 0.7206 - val_acc: 0.5813 - val_loss: 0.7127
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc: 0.5437 - loss: 0.7092 - val_acc: 0.5875 - val_loss: 0.7040
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - acc: 0.5594 - loss: 0.7012 - val_acc: 0.5188 - val_loss: 0.6979
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc: 0.5703 - loss: 0.6951 - val_acc: 0.5938 - val_loss: 0.6920
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc: 0.6125 - loss: 0.6795 - val_acc: 0.6375 - val_loss: 0.6518
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - acc: 0.7281 - loss: 0.5597 - val_acc: 0.6812 - val_loss: 0.5886
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - acc: 0.8469 - loss: 0.3780 - val_acc: 0.7188 - val_loss: 0.6577
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - acc: 0.8922 - loss: 0.2881 - 

We get val accuracy of 76% which is nice. but we can still improve it.